In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import os

import numpy as np
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

import h5py 

import pandas
import scipy.ndimage

# Load ONC DB and mask files of interest

In [ ]:
# Check if DB exits, if not load it, but update it anyway
if os.path.exists(strawb.Config.pandas_file_sync_db):
    db = strawb.SyncDBHandler()  # loads the db from disc
else:
    db = strawb.SyncDBHandler(load_db=False)  # loads the db from ONC server

db.load_onc_db_update(output=True, save_db=True)

In [ ]:
# mask by device and data-product
mask = db.dataframe['deviceCode'] == 'TUMMUONTRACKER001'
mask &= db.dataframe.dataProductCode == 'MTSD'  # see SyncDBHandler.sensor_mapping

In [ ]:
db.dataframe.dateTo.max()

In [ ]:
# show some masked entries
db.dataframe[mask].iloc[-5:]

# Download a specific file, needs some minutes if file isn't synced

In [ ]:
file_list = ['TUMMUONTRACKER001_20220731T001146.146Z-SDAQ-MUON.hdf5']
db_i = db.get_files_from_names(file_list)
db_i

# Load the file

In [ ]:
muon = strawb.MuonTracker(file_list[0])

## show file version - 4 has tot data, which we need

In [ ]:
muon.file_handler.file_version

## Show data structure

In [ ]:
# all have the same shape -> entry i represents one event
print(f'tot_time   : {muon.file_handler.tot_time.shape}')
print(f'tot_time_ns: {muon.file_handler.tot_time_ns.shape}')
print(f'tot_channel: {muon.file_handler.tot_channel.shape}')
print(f'tot_tot    : {muon.file_handler.tot_tot.shape}')

In [ ]:
# show the first n events
n = 5

# Absolute Timestamp in seconds [s] with resolution [ns]
print(f'tot_time   : {muon.file_handler.tot_time[:n]}')

# Absolute Timestamp in seconds [s] with resolution [ns], converted to dateformat
print(f'tot_time   : {muon.file_handler.tot_time.asdatetime()[:n]}')

# TRB Timestamp in [ns] - not absolute therefore most precise
print(f'tot_time_ns: {muon.file_handler.tot_time_ns[:n]}')

# TRB Channel of the event
print(f'tot_channel: {muon.file_handler.tot_channel[:n]}')

# TRB time over threshold in [ns]
print(f'tot_tot    : {muon.file_handler.tot_tot[:n]}')

# Load and Cut Data
It seems, the TRB has an overflow at 2750 in both time data. (can be corrected with np.unwrap)

In [ ]:
tot = muon.file_handler.tot_tot[:]

# some tot values are very high ~1e12, exclude them here or not
if True:
    max_tot = 1 * 1e3 # cut at 1us - [ns]
    mask = (tot<max_tot) & (tot > 0)
    print(f'exclude {np.sum(~mask)} events')
else:
    mask = np.ones_like(tot, dtype=bool)

trb_overflow = 2750.

# time in seconds, time_masked since epoch (1.1.1970) and time_ns_masked TRB internal (more precise)
time_masked = np.unwrap(muon.file_handler.tot_time[mask], period=trb_overflow)
time_ns_masked = np.unwrap(muon.file_handler.tot_time_ns[mask], period=trb_overflow)

# tot in nano-seconds
tot_masked = tot[mask]

# channel id
channel_masked = muon.file_handler.tot_channel[mask]

# the time isn't sorted correctly, do it here
index_sort = np.argsort(time_ns_masked)

time_masked = time_masked[index_sort]
time_ns_masked  = time_ns_masked[index_sort]
channel_masked = channel_masked[index_sort]
tot_masked = tot_masked[index_sort]

# free RAM - parameter not needed
del tot, mask, index_sort

# Plot

In [ ]:
# timeline of events

plt.figure()

n = 1000
sc = plt.scatter(strawb.tools.asdatetime(time_masked)[:n],
            tot_masked[:n],
            c=channel_masked[:n])

plt.colorbar(sc, label='Channel')

ax = plt.gca()
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator())
)
ax.set_axisbelow(True)  # show grid behind scatter

plt.ylabel('ToT [ns]')
plt.xlabel('Date [s]')
plt.grid()
plt.tight_layout()

## Show short ToT's

In [ ]:
# gen. hist
counts, edges = np.histogram(tot_masked, bins=1000)

# plot hist
plt.figure()
plt.stairs(counts, edges=edges)
plt.xlabel('ToT [ns]')
plt.ylabel('Counts')
plt.yscale('log')
plt.grid()
plt.tight_layout()

In [ ]:
plt.figure()
plt.hist(np.diff(time_ns_masked), bins=1000)
plt.yscale('log')
plt.xlabel('Delta time between events [s]')
plt.ylabel('Counts')
plt.grid()
plt.tight_layout()

# Event builder
Detect and build events. A event is defined as set of tot-events where the timestamps between the singel tot-events isn't greater as a limit (`dt_max`). Muons are traveling with ~c and 1ns corresbonds to 0.3m distance. The mountracker has a diameter of 13", so ~0.3m.    

In [ ]:
# Test values code for event builder
x = np.array([0, 1, 2, 2.1, 2.2, 2.3, 3, 3.1, 4, 5])
target = np.array([0, 0, 1,   1,   1,   1, 2, 2, 0, 0])

m = np.diff(x) < .2
label_diff, num_features = scipy.ndimage.label(m)
# print('label: ', label)

label = np.zeros(x.shape, dtype=label_diff.dtype)
label[1:][label_diff!=0] = label_diff[label_diff!=0]
label[:-1][label_diff!=0] = label_diff[label_diff!=0]

del label_diff, m

print('label: ', label)
label - target

## Build events for dataset

In [ ]:
import scipy.ndimage

dt_max = 5e-9  # in seconds, e.g. 1e-8=10ns <-> 3m for light (in vacuum)

m = np.diff(time_ns_masked) < dt_max
label_diff, num_features = scipy.ndimage.label(m)

label = np.zeros(time_ns_masked.shape, dtype=label_diff.dtype)
label[1:][label_diff!=0] = label_diff[label_diff!=0]
label[:-1][label_diff!=0] = label_diff[label_diff!=0]
del label_diff, m

# get the slices for each event
slices = scipy.ndimage.measurements.find_objects(label)

# get the number of channels in a event
label_u, counts = np.unique(label, return_counts=True)
counts_events = counts[label_u>0]

### Show one Event

In [ ]:
# show one Event, here the one with maximum active channels in the dataset
# get
l_i = np.argwhere(counts_events.max() == counts).flatten()
l_i -= 1  # because find_objects ignores label=0
slice_i = slices[l_i[0]]
time_ns_masked[slice_i] - time_ns_masked[slice_i][0], channel_masked[slice_i], tot_masked[slice_i]

# PLOT

In [ ]:
# plot the active channels per event
plt.figure()

plt.hist(counts_events, bins=np.arange(2, counts_events.max()+1)-.5)
plt.yscale('log')
plt.xlabel('Delta time between events [s]')
plt.ylabel('Counts')

plt.grid()
plt.tight_layout()

# Build events with multiplicity `length`

In [ ]:
import scipy.stats
import tqdm.notebook

In [ ]:
length = 2
args_select = np.argwhere(counts[1:]==length).flatten()

event_channels = np.zeros((len(args_select), length), dtype=np.uint8)
event_tot = np.zeros((len(args_select), length), dtype=float)
event_t_ns = np.zeros((len(args_select), length), dtype=float)

for i, s_i in tqdm.notebook.tqdm(enumerate(args_select), total=len(args_select)):
    slice_i = slices[s_i]
    event_channels[i] = channel_masked[slice_i]
    event_tot[i] = tot_masked[slice_i]
    event_t_ns[i] = time_ns_masked[slice_i]

In [ ]:
# print the unique pairs with count
u, c = np.unique(np.sort(event_channels, axis=-1), axis=0, return_counts=True)
print('Counts; Channel_i')
np.append(c[None], u.T, axis=0).T

In [ ]:
binned_stat = scipy.stats.binned_statistic_dd(np.sort(event_channels),
                                              np.ones(len(event_channels)),
                                              bins=[np.arange(0, 18)-.5]*2,
                                              statistic='count')

statistic = np.ma.masked_less(binned_stat.statistic.copy(), 1)

In [ ]:
statistic_i = statistic.copy()
statistic_i[statistic_i>1e4] = 1e4

cb = plt.matshow(np.log(statistic_i.filled(np.nan)), vmin=1)
plt.colorbar(cb, shrink=.6, label='log(counts)')
plt.xlabel('channel')
plt.ylabel('channel')
plt.grid(lw=.5)
plt.xlim(.5, 16.5)
plt.ylim(16.5, .5)
plt.xticks(ticks=np.arange(1,17))
plt.yticks(ticks=np.arange(1,17))

plt.show()